<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString, Point
from shapely.ops import unary_union
import folium
from geopy.distance import geodesic

In [88]:
# 'trips.xlsx' is uploaded to Colab environment
df_trips = pd.read_excel('trips.xlsx', header=0)  # header=0 means the first row is the header
# df_trips.sample(2)

In [89]:
# Split the 'Coordinates' column into 'Longitude' and 'Latitude'
df_trips[['Latitude', 'Longitude']] = df_trips['Сoordinates'].str.replace('°', '').str.replace(',', '.').str.split('. ', expand=True)

# Convert 'Day_time' to datetime objects
df_trips['Day_time'] = pd.to_datetime(df_trips['Day_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')

# df_trips['Ignition'] = df_trips['Ignition'].str.replace('-', '0').str.replace('+', '1')

df_trips = df_trips.sort_values(['Car_ID', 'Day_time'])  # Сортируем по времени

# Преобразование поездок в геометрию
df_trips['geometry'] = [Point(lon, lat) for lon, lat in zip(df_trips['Longitude'], df_trips['Latitude'])]
df_trips = gpd.GeoDataFrame(df_trips, geometry='geometry', crs="EPSG:4326")

df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Latitude,Longitude,geometry
3036,77403 (М947ОК67) Renault Logan,2025-02-18 16:26:38,"55,76599°, 37,822656°",13,+,55.76599,37.822656,POINT (37.82266 55.76599)
2507,77403 (М947ОК67) Renault Logan,2025-02-18 07:06:43,"55,76696°, 37,823241°",19,+,55.76696,37.823241,POINT (37.82324 55.76696)


In [90]:
# Загрузка данных о Выделенных полосах
gdf_bus_lanes = gpd.read_file("bus-lanes.geojson")
# Выделяем линии Москвы
gdf_bus_lanes = gdf_bus_lanes[gdf_bus_lanes['short_name'] == 'Москва']
# Устанавливаем корректную CRS
gdf_bus_lanes = gdf_bus_lanes.set_crs("EPSG:4326")
# gdf_bus_lanes.sample(2)

,admin_level,index_right,lanes,length,name,place,population,short_name,type,lanes_length,geometry
820,None,NaN,1,375.369098,Москва,None,12655050.0,Москва,None,375.369098,"LINESTRING (37.42624 55.62274, 37.42269 55.62003)"
583,None,NaN,2,18339.649739,Москва,None,12655050.0,Москва,None,36679.299479,"LINESTRING (37.41046 55.54383, 37.39231 55.536..."


In [91]:
# Преобразуем CRS в метры (EPSG:3857)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=3857)
# Делаем buffer на 50 метров
gdf_bus_lanes["geometry"] = gdf_bus_lanes.geometry.buffer(50)
# Возвращаем обратно в географические координаты (EPSG:4326)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=4326)
# gdf_bus_lanes.sample(2)

,admin_level,index_right,lanes,length,name,place,population,short_name,type,lanes_length,geometry
498,None,NaN,2,5484.587215,Москва,None,12655050.0,Москва,None,10969.174429,"POLYGON ((37.56532 55.78555, 37.53078 55.80063..."
275,6,97.0,1,165.913504,городской округ Ростов-на-Дону,None,1137704.0,Ростов-на-Дону,None,165.913504,"POLYGON ((39.67586 47.21131, 39.6759 47.21131,..."


In [79]:
# Объединяем все POLYGON в один
bus_lanes_union = unary_union(gdf_bus_lanes.geometry)

# Проверяем пересечение точек с объединенной зоной
df_trips["on_bus_lane"] = df_trips["geometry"].apply(lambda point: bus_lanes_union.intersects(point))
# df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Latitude,Longitude,geometry,on_bus_lane
3309,77403 (М947ОК67) Renault Logan,2025-02-18 19:12:44,"55,754048°, 37,815136°",7,+,55.754048,37.815136,POINT (37.81514 55.75405),False
2147,77403 (М947ОК67) Renault Logan,2025-02-18 01:03:58,"55,76402°, 37,832526°",45,+,55.76402,37.832526,POINT (37.83253 55.76402),False


In [93]:
# Вариант векторной проверки
# Объединяем все POLYGON в один
bus_lanes_union = unary_union(gdf_bus_lanes.geometry)

# Векторизованная проверка пересечений
df_trips["on_bus_lane"] = gpd.GeoSeries(df_trips["geometry"]).intersects(bus_lanes_union)

In [94]:
# Анализ скорости на участках
df_trips["next_time"] = df_trips.groupby('Car_ID')['Day_time'].shift(-1)
df_trips["next_lon"] = df_trips.groupby('Car_ID')['Longitude'].shift(-1)
df_trips["next_lat"] = df_trips.groupby('Car_ID')['Latitude'].shift(-1)
df_trips["next_on_bus_lane"] = df_trips.groupby('Car_ID')['on_bus_lane'].shift(-1)

# Вычисляем расстояние между последовательными точками в км
# Создаем два массива для координат
coords1 = np.column_stack((df_trips['Latitude'], df_trips['Longitude']))
coords2 = np.column_stack((df_trips['next_lat'], df_trips['next_lon']))

# Функция для вычисления расстояний
def calculate_distances(coords1, coords2):
    return np.array([geodesic((lat1, lon1), (lat2, lon2)).meters / 1000
                     if not pd.isna(lat2) else np.nan
                     for (lat1, lon1), (lat2, lon2) in zip(coords1, coords2)])

# Применяем функцию для всей таблицы
df_trips['distance'] = calculate_distances(coords1, coords2)

# Вычисляем время между последовательными точками, часов, Заменяем NaN на None
time_diff = (df_trips["next_time"] - df_trips['Day_time']).dt.total_seconds() / 3600
df_trips["time"] = time_diff.where(df_trips["next_time"].notna(), None)

# Проверяем, чтобы значения в столбцах "distance" и "time" были валидными, и вычисляем скорость
df_trips["speed"] = np.where(
    (pd.notnull(df_trips["distance"])) & (df_trips["time"].notnull()) & (df_trips["time"] != 0),
    df_trips["distance"] / df_trips["time"],
    None
)

In [ ]:
df_trips.iloc[236:239]

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Longitude,Latitude,geometry,on_bus_lane,next_time,next_lon,next_lat,distance,time,speed
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,55.767596. 37.83425,15,1,55.767596,37.83425,POINT (55.7676 37.83425),False,2025-02-17 06:53:31,55.76751,37.834183,0.010612,0.001111,9.550852
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,55.76751. 37.834183,7,1,55.76751,37.834183,POINT (55.76751 37.83418),False,2025-02-17 06:53:32,55.767491,37.834136,0.005478,0.000278,19.721727
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,55.767491. 37.834136,14,1,55.767491,37.834136,POINT (55.76749 37.83414),False,2025-02-17 06:53:33,55.767483,37.834058,0.008686,0.000278,31.269875


In [95]:
# Оценка скорости на выделенных полосах
avg_speed_bus_lane = 50

# Рассчитываем предсказанное время для каждой строки
df_trips['predicted_time'] = np.where(
    (df_trips["on_bus_lane"] & df_trips["next_on_bus_lane"]),
    np.minimum(df_trips['time'], df_trips['distance'] / avg_speed_bus_lane),
    df_trips['time']
)

In [ ]:
df_trips.iloc[236:239]

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition,Latitude,Longitude,geometry,on_bus_lane,next_time,next_lon,next_lat,next_on_bus_lane,distance,time,speed,predicted_time
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,55.767596. 37.83425,15,+,55.767596,37.83425,POINT (37.83425 55.7676),False,2025-02-17 06:53:31,37.834183,55.76751,False,0.010458,0.001111,9.412056,0.001111
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,55.76751. 37.834183,7,+,55.76751,37.834183,POINT (37.83418 55.76751),False,2025-02-17 06:53:32,37.834136,55.767491,False,0.003630,0.000278,13.068405,0.000278
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,55.767491. 37.834136,14,+,55.767491,37.834136,POINT (37.83414 55.76749),False,2025-02-17 06:53:33,37.834058,55.767483,False,0.004976,0.000278,17.914238,0.000278


In [96]:
# время в движении, часов
total_time = df_trips.loc[df_trips['speed'] > 0, 'time'].sum()
# время в движении с использованием выделенных полос, часов
total_predicted_time = df_trips.loc[df_trips['speed'] > 0, 'predicted_time'].sum()
total_time_saved = total_time - total_predicted_time

print(f"Общее время в движении: {total_time :.2f} часов")
print(f"Общая экономия времени: {total_time_saved :.2f} часов")
print(f"% экономии времени: {total_time_saved / total_time  * 100  :.2f} %")

Общее время в движении: 6.98 часов
Общая экономия времени: 0.46 часов
% экономии времени: 6.54 %
